In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
from sklearn.preprocessing import LabelEncoder

In [26]:
import os
import glob
import time
import holidays
import numpy as np
import pandas as pd
import datetime as datetime
from sklearn.cluster import DBSCAN
from geopy.distance import distance
from multiprocessing import Process, Manager
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import haversine_distances

import plotly.express as px
import plotly.graph_objects as go

us_holidays = holidays.country_holidays('US')

directory = '/content/drive/MyDrive/Thesis/'


data_dir = directory + "Data/"

# Read CSV

In [7]:
from shapely.geometry import Point, Polygon

# Define the polygon for Manhattan (approximate boundaries)
manhattan_polygon = Polygon([
    (-73.926300, 40.877671),
    (-73.922994, 40.877227),
    (-73.920024, 40.875224),
    (-73.916280, 40.874640),
    (-73.914403, 40.874510),
    (-73.911626, 40.873464),
    (-73.910370, 40.872291),
    (-73.909903, 40.870698),
    (-73.914040, 40.864194),
    (-73.916533, 40.860966),
    (-73.925260, 40.850602),
    (-73.929904, 40.843885),
    (-73.933885, 40.835120),
    (-73.933885, 40.828478),
    (-73.933145, 40.819519),
    (-73.933528, 40.809380),
    (-73.928220, 40.801866),
    (-73.928629, 40.795008),
    (-73.938756, 40.784106),
    (-73.940773, 40.774839),
    (-73.951841, 40.762962),
    (-73.968080, 40.745712),
    (-73.969998, 40.721783),
    (-73.976165, 40.708818),
    (-73.993777, 40.706617),
    (-74.009752, 40.698527),
    (-74.019027, 40.699648),
    (-74.022106, 40.705039),
    (-74.011532, 40.755631),
    (-74.001283, 40.775638),
    (-73.931169, 40.879602),
    (-73.926300, 40.877671)   # Closing the polygon
])

def is_in_manhattan(lat, lon, polygon):
    point = Point(lon, lat)
    return polygon.contains(point)


In [8]:
raw_data = pd.DataFrame()

months = ["01", "03", "07", "11", "12"]

for subdir, _, _ in os.walk(data_dir):
  # Use glob to find all CSV files in the current directory
  csv_files = glob.glob(os.path.join(subdir, '*.csv'))

  # Read each CSV file and store the dataframe in the list
  for file in csv_files:
    if "2022" not in file and any(month in file for month in months):
      print(file)
      df = pd.read_csv(file)


      print(df.shape)
      manhattan_df = df[df.apply(lambda row: is_in_manhattan(
          row['end_lat'],
          row['end_lng'],
          manhattan_polygon), axis=1)]

      manhattan_df = manhattan_df[manhattan_df.apply(lambda row: is_in_manhattan(
          row['start_lat'],
          row['start_lng'],
          manhattan_polygon), axis=1)]

      print(manhattan_df.shape)
      raw_data = pd.concat([raw_data, manhattan_df])


/content/drive/MyDrive/Thesis/Data/202401-citibike-tripdata_1.csv


<ipython-input-8-5c5db61997dc>:13: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


(1000000, 13)
(659739, 13)
/content/drive/MyDrive/Thesis/Data/202401-citibike-tripdata_2.csv


<ipython-input-8-5c5db61997dc>:13: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


(954376, 13)
(582251, 13)
/content/drive/MyDrive/Thesis/Data/2023-citibike-tripdata/12_December/202312-citibike-tripdata_2.csv


...

(1000000, 13)
(686145, 13)
/content/drive/MyDrive/Thesis/Data/2023-citibike-tripdata/3_March/202303-citibike-tripdata_3.csv
(194214, 13)
(141111, 13)
/content/drive/MyDrive/Thesis/Data/2023-citibike-tripdata/1_January/202301-citibike-tripdata_1.csv


<ipython-input-8-5c5db61997dc>:13: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


(1000000, 13)
(644678, 13)
/content/drive/MyDrive/Thesis/Data/2023-citibike-tripdata/1_January/202301-citibike-tripdata_2.csv


<ipython-input-8-5c5db61997dc>:13: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


(855201, 13)
(593280, 13)


In [9]:
del df
del manhattan_df
raw_data['started_at'] = pd.to_datetime(raw_data['started_at'])
raw_data['ended_at'] = pd.to_datetime(raw_data['ended_at'])
raw_data['duration_sec'] = (raw_data['ended_at'] - raw_data['started_at'])/ pd.Timedelta(seconds=1)


In [10]:
raw_data

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration_sec
0,58F2CA262B50E256,classic_bike,2024-01-25 20:39:09,2024-01-25 20:44:07,Broadway & E 14 St,5905.12,Ave A & E 11 St,5703.13,40.734546,-73.990741,40.728547,-73.981759,member,298.0
4,23EE6A8979C333B1,classic_bike,2024-01-27 09:55:39,2024-01-27 10:00:48,E 16 St & Irving Pl,5938.11,E 14 St & 1 Ave,5779.10,40.735367,-73.987974,40.731393,-73.982867,member,309.0
6,791FB703B226E19B,electric_bike,2024-01-08 21:24:49,2024-01-08 21:32:29,Gansevoort St & Hudson St,6072.16,Ave A & E 11 St,5703.13,40.739378,-74.004906,40.728547,-73.981759,member,460.0
10,49DAE8A1405C6950,classic_bike,2024-01-12 14:05:20,2024-01-12 14:10:28,E 16 St & Irving Pl,5938.11,Ave A & E 11 St,5703.13,40.735367,-73.987974,40.728547,-73.981759,member,308.0
12,27E4B8862093C0C9,classic_bike,2024-01-08 19:23:28,2024-01-08 19:26:54,1 Ave & E 18 St,5854.09,Ave A & E 11 St,5703.13,40.733812,-73.980544,40.728547,-73.981759,member,206.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855194,9B04B6171509F243,classic_bike,2023-01-29 17:08:08,2023-01-29 17:28:51,W Broadway & Spring St,5569.06,W 21 St & 6 Ave,6140.05,40.724947,-74.001659,40.741740,-73.994156,member,1243.0
855195,C6324FCDB7233616,classic_bike,2023-01-03 08:58:19,2023-01-03 09:10:53,E 14 St & 1 Ave,5779.1,W 21 St & 6 Ave,6140.05,40.731393,-73.982867,40.741740,-73.994156,member,754.0
855197,ABE3E9D3F081A13E,classic_bike,2023-01-03 17:47:28,2023-01-03 17:52:21,E 115 St & Madison Ave,7599.02,Park Ave & E 124 St,7682.01,40.798944,-73.944846,40.804555,-73.939686,member,293.0
855198,8F1B225FFE447DD6,classic_bike,2023-01-12 20:18:52,2023-01-12 20:21:31,E 14 St & 1 Ave,5779.1,E 12 St & Ave C,5616.08,40.731402,-73.982845,40.727243,-73.976831,member,159.0


# Finding Standard Stations

In [11]:
start_stations = raw_data[[
    "start_station_id",
    'start_lat',
    "start_lng"]]\
.sort_values(['start_lat', "start_lng"])\
.drop_duplicates(subset=['start_lat','start_lng'])\
.dropna().drop_duplicates(subset=['start_station_id'])\
.query("start_lat != 0").reset_index(drop=True).rename(
    columns={
        "start_station_id": "station_id",
        'start_lat': "station_latitude",
        "start_lng": "station_longitude"
  })

end_stations = raw_data[["end_station_id",
                           'end_lat',
                           "end_lng"]]\
.sort_values(['end_lat', "end_lng"])\
.drop_duplicates(subset=['end_lat','end_lng'])\
.dropna().drop_duplicates(subset=['end_station_id'])\
.query("end_lat != 0").reset_index(drop=True).rename(
    columns={"end_station_id": "station_id",
            'end_lat': "station_latitude",
            "end_lng": "station_longitude"
        })

In [12]:
standard_stations = pd.concat([start_stations, end_stations]).sort_values(
    ['station_latitude', "station_longitude"]).drop_duplicates(subset=['station_id'])

# Clustering

In [16]:
points_in_radians = standard_stations[['station_latitude','station_longitude']].apply(np.radians).values
distances_in_km = haversine_distances(points_in_radians) * 6371

In [17]:
distance_matrix = distances_in_km

clustering_areas = DBSCAN(eps= 0.1, min_samples=20)
clustering_stations = DBSCAN(eps= 0.0045, min_samples=1)
scaler = MinMaxScaler()
df_temp = standard_stations.copy()
df_temp[['station_latitude', 'station_longitude']] = \
                            scaler.fit_transform(df_temp[['station_latitude', 'station_longitude']])
clusters_areas = clustering_areas.fit_predict(df_temp[['station_latitude', 'station_longitude']])

clusters_stations = clustering_stations.fit_predict(df_temp[['station_latitude', 'station_longitude']])

In [18]:
del df_temp

In [19]:
standard_stations['clusters'] = clusters_areas
standard_stations['cluster_stations'] = clusters_stations

centroids = standard_stations.groupby("clusters")[['station_latitude', 'station_longitude']].mean()
centroids = centroids.reset_index().rename(columns={
    "station_latitude": "station_latitude_centroid",
    "station_longitude": "station_longitude_centroid",
})

centroid_stations = standard_stations.groupby("cluster_stations")[['station_latitude', 'station_longitude']].mean()
centroid_stations = centroid_stations.reset_index().rename(columns={
    "station_latitude": "station_latitude_centroid_stations",
    "station_longitude": "station_longitude_centroid_stations",
})

In [27]:
fig = px.scatter_mapbox(standard_stations, lat='station_latitude', lon='station_longitude',
                        color = "clusters",
                        mapbox_style="open-street-map",
                        hover_name="station_id",
                       )

fig.add_trace(go.Scattermapbox(
        lat=centroids['station_latitude_centroid'],
        lon=centroids['station_longitude_centroid'],
        mode='markers',
        hoverinfo='text',
        name="Cluster Centroids",
        marker=go.scattermapbox.Marker(
            size=15,
            color='rgb(0, 0, 0)',
            opacity=0.7
        ),
    ))

fig.update_geos(fitbounds="locations")
fig.show()

In [28]:
fig = px.scatter_mapbox(standard_stations, lat='station_latitude', lon='station_longitude',
                        color = "cluster_stations",
                        mapbox_style="open-street-map",
                        hover_name="station_id",
                       )

fig.add_trace(go.Scattermapbox(
        lat=centroid_stations['station_latitude_centroid_stations'],
        lon=centroid_stations['station_longitude_centroid_stations'],
        mode='markers',
        hoverinfo='text',
        name="Cluster Centroids",
        marker=go.scattermapbox.Marker(
            size=15,
            color='rgb(0, 0, 0)',
            opacity=0.7
        ),
    ))

fig.update_geos(fitbounds="locations")
fig.show()

In [36]:
centroids.to_csv(data_dir+"centroids.csv")

In [37]:
centroid_stations.to_csv(data_dir+"station_centroids.csv")

In [38]:
standard_stations.to_csv(data_dir+"standard_centroids.csv")